# Text Splitting
After Text-loading from the provided document the next phase of the project will be the **Text Splitting**.<br>
**Text Splitting** is done in order to divide the loaded text from document into smaller chunks. The reason we do this is because of the token-size-limits of llms. The limits are set because processing more tokens demands more memory, time, and processing power, making it crucial to balance efficiency with performance.
<br>
That is why splitting of text is required to reduce the size of text such that it should remain within token limit.
<br>
<br>
Text splitting of the loaded text is done in 3 steps:
1.   Splits
2.   Merging
3.   Overlapping

### Splits:
Loaded text is divided in to few chunks called splits.
<br>
### Merging:
Once the splits are created these are further merged. It is done because we need to ensure that these splits are within the token limit as some chunks might be of the smaller size.
<br>
### Overlapping:
After splits are merged we need to add some portion of predecessor split in order to maintain the context as it may cause ambiguity.
<br>
<br>
Let us see some methods for text Splitting
<br>
***

## Manual Approach

In [ ]:
text = """Japan, island country lying off the east coast of Asia. It consists of a great string of islands in a northeast-southwest arc that stretches for approximately 1,500 miles (2,400 km) through the western North Pacific Ocean. Nearly the entire land area is taken up by the country’s four main islands; from north to south these are Hokkaido (Hokkaidō), Honshu (Honshū), Shikoku, and Kyushu (Kyūshū). Honshu is the largest of the four, followed in size by Hokkaido, Kyushu, and Shikoku. In addition, there are numerous smaller islands, the major groups of which are the Ryukyu (Nansei) Islands (including the island of Okinawa) to the south and west of Kyushu and the Izu, Bonin (Ogasawara), and Volcano (Kazan) islands to the south and east of central Honshu. The national capital, Tokyo (Tōkyō), in east-central Honshu, is one of the world’s most populous cities.
Mount FujiMount Fuji from its northern side, Yamanashi prefecture, east-central Honshu, Japan.
The Japanese landscape is rugged, with more than four-fifths of the land surface consisting of mountains. There are many active and dormant volcanoes, including Mount Fuji (Fuji-san), which, at an elevation of 12,388 feet (3,776 meters), is Japan’s highest mountain. Abundant precipitation and the generally mild temperatures throughout most of the country have produced a lush vegetation cover and, despite the mountainous terrain and generally poor soils, have made it possible to raise a variety of crops. Japan has a large and, to a great extent, ethnically homogeneous population, which is heavily concentrated in the low-lying areas along the Pacific coast of Honshu.


Quick Facts
Japan
Head Of Government: Prime Minister: Shigeru Ishiba
Capital: Tokyo
Population: (2025 est.) 123,105,000
Currency Exchange Rate: 1 USD equals 157.826 Japanese yen
Form Of Government: constitutional monarchy with a national Diet consisting of two legislative houses (House of Councillors [242]; House of Representatives [475])


Kinkaku Temple
Kinkaku TempleThe Kinkaku Temple (Golden Pavilion) in Kyōto, Japan, was originally built in the 15th century; the present structure dates to the 1950s.
Akihabara district in Tokyo
Akihabara district in TokyoTrain of the Sōbu Line passing through Akihabara, a district of Tokyo renowned for its many discount stores selling electric and electronic products.
Complexity and contrast are the keynotes of life in Japan—a country possessing an intricate and ancient cultural tradition yet one that, since 1950, has emerged as one of the world’s most economically and technologically advanced societies. Heavy emphasis is placed on education, and Japan is one of the world’s most literate countries. Tension between old and new is apparent in all phases of Japanese life. A characteristic sensitivity to natural beauty and a concern with form and balance are evident in such cities as Kyōto and Nara, as well as in Japan’s ubiquitous gardens. Even in the countryside, however, the impact of rapid Westernization is evident in many aspects of Japanese life. The agricultural regions are characterized by low population densities and well-ordered rice fields and fruit orchards, whereas the industrial and urbanized belt along the Pacific coast of Honshu is noted for its highly concentrated population, heavy industrialization, and environmental pollution.


Humans have occupied Japan for tens of thousands of years, but Japan’s recorded history begins only in the 1st century bce, with mention in Chinese sources. Contact with China and Korea in the early centuries ce brought profound changes to Japan, including the Chinese writing system, Buddhism, and many artistic forms from the continent. The first steps at political unification of the country occurred in the late 4th and early 5th centuries ce under the Yamato court. A great civilization then developed first at Nara in the 8th century and then at Heian-kyō (now Kyōto) from the late 8th to the late 12th century. The seven centuries thereafter were a period of domination by military rulers culminating in near isolation from the outside world from the early 17th to the mid-19th century.
The reopening of the country ushered in contact with the West and a time of unprecedented change. Japan sought to become a modern industrialized nation and pursued the acquisition of a large overseas empire, initially in Korea and China. By late 1941 this latter policy caused direct confrontation with the United States and its allies and to defeat in World War II (1939–45). Since the war, however, Japan’s spectacular economic growth—one of the greatest of any nation in that period—brought the country to the forefront of the world economy. It now is one of the world’s foremost manufacturing countries and traders of goods and is a global financial leader.
"""

#Creating splits with by string slicing, considering token limit is 60
text[0:60]

'Japan, island country lying off the east coast of Asia. It c'

The problem with the manual string slicing method is that we can see that some words contained in text got cut off in the output. Because of this problem, it may cause issue in model as it may consider single sliced word as two distinct words. It can be further solved by using built in split function and creating array of those splits.

In [66]:
words = text.split(" ")
splits = []

#considering the limit of token is 100
limit = 100

s = ""
for word in words:
    s += word + " "
    if len(s)>limit:
        splits.append(s)
        s = ""

print(splits[0])

Japan, island country lying off the east coast of Asia. It consists of a great string of islands in a 


Through this method we can create splits within token limit but it is tidious task, also we may need to experiment with various delimiters ensuring that split does'nt exceed the token limit.
<br>
Therefore **langchain** provide and efficient way to split loaded text through **text-splitters**

## Langchain Text-splitters
There are several text-splitters in langchain some of these are:

In [67]:
#Character text-splitter
from langchain.text_splitter import CharacterTextSplitter as cts

#Character text-splitter consists of several argument some of them are separator, chunk_size and chunk_overlap.
ts = cts(
    separator="\n",
    #consider token size as 100
    chunk_size = 100,
    #considering there is no need for context for model
    chunk_overlap = 0
)

splits = ts.split_text(text)
len(splits)

13

In [68]:
for split in splits:
  print(len(split))

861
94
673
83
93
162
14
151
27
176
992
793
661


The problem with this approach is that without even overlapping some splits exceed the specified limit. This is because of the seperator ***/n*** we are using in the function which i s new-line character and some potriton of the text doesn't have the new-line character.
<br>For example:

In [69]:
splits[11]

'Humans have occupied Japan for tens of thousands of years, but Japan’s recorded history begins only in the 1st century bce, with mention in Chinese sources. Contact with China and Korea in the early centuries ce brought profound changes to Japan, including the Chinese writing system, Buddhism, and many artistic forms from the continent. The first steps at political unification of the country occurred in the late 4th and early 5th centuries ce under the Yamato court. A great civilization then developed first at Nara in the 8th century and then at Heian-kyō (now Kyōto) from the late 8th to the late 12th century. The seven centuries thereafter were a period of domination by military rulers culminating in near isolation from the outside world from the early 17th to the mid-19th century.'

We can change the separator to "." or " ", but it will increase the split quantity. For some instances Character Text-Splitter might work. But for llms having strict token limit it can cause issue. There for we may need to work on multiple separator recursively in-order to achieve the efficient splits.
<br>
For such cases we can use the Recursive Text-Splitter

In [70]:
# Recursive Text-Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter as rts

#Recurive text-splitter consists of similar arguments as Character Text-Splitter with only difference of separators
#In separator we need to add the list of Separators.
rs = rts(
    separators=["\n\n","\n"," "],
    chunk_size = 200,
    chunk_overlap = 0
)

splits = rs.split_text(text)
len(splits)

28

In [71]:
for split in splits:
  print(len(split))

193
192
194
196
82
94
196
197
199
78
177
177
179
176
187
195
195
188
198
24
198
195
199
198
199
191
190
78


Understanding how it works

In [80]:
#let us take another sample text for this
text2 = """Japan, island country lying off the east coast of Asia.
It consists of a great string of islands in a northeast-southwest arc that stretches for approximately 1,500 miles (2,400 km) through the western North Pacific Ocean. Nearly the entire land area is taken up by the country’s four main islands; from north to south these are Hokkaido (Hokkaidō), Honshu (Honshū), Shikoku, and Kyushu (Kyūshū).
Honshu is the largest of the four, followed in size by Hokkaido, Kyushu, and Shikoku. In addition, there are numerous smaller islands, the major groups of which are the Ryukyu (Nansei) Islands (including the island of Okinawa) to the south and west of Kyushu and the Izu, Bonin (Ogasawara), and Volcano (Kazan) islands to the south and east of central Honshu.
The national capital, Tokyo (Tōkyō), in east-central Honshu, is one of the world’s most populous cities.
Mount FujiMount Fuji from its northern side, Yamanashi prefecture, east-central Honshu, Japan."""

# assuming the provided list of separator is ["/n","."]. There Recusrive text-splitter first creates split from "/n" lets just say its split1.
split1 = text2.split("\n")
for split in split1:
  print(len(split))

56
341
360
104
94


In [81]:
# Once the split is created, now it check the length of each split. If the size of split exceeds the chunks size it will do next recursion.
# Since the size of 2nd split i.e. split1[1] is greater than specified size, lets just say 200 creates another split from "." separator lets say its split2.
split2 = split1[1].split(".")
for split in split2:
  print(len(split))

165
173
1
